In [1]:
# Weifan Lin
from operator import add

In [2]:
# method 1
rdd = sc.textFile('citibike.csv')

def mapper1(records):
    for line in records:
        yield (int(line.split(',')[2])/60, 1)

def mapper2(durationCounts):
    li = []
    for i in durationCounts:
        li.append(i)
    yield li

def computeMedian(list1, list2):
    li = list1+list2
    totalCount = 0
    medianCount = 0
    for duration, count in li:
        totalCount += count
    for duration, count in li:
        medianCount += count
        if medianCount > totalCount/2:
            yield duration
            break
    
durationCount = rdd.mapPartitions(mapper1).reduceByKey(add).sortByKey()
median = list(durationCount.mapPartitions(mapper2).reduce(computeMedian))
median # median is 8

[8]

In [3]:
# method 2
lines = sc.textFile('citibike.csv')
minutes = lines.map(lambda x: (int(x.split(',')[2])/60,))
df = sqlContext.createDataFrame(minutes, ('minutes', )).cache()
df.registerTempTable('records')
sqlContext.sql('select percentile(minutes, 0.5) from records').collect()

[Row(_c0=8.0)]